In [1]:
from transformers import AutoTokenizer, RobertaForSequenceClassification, AutoModelForSequenceClassification
from tqdm import tqdm
import torch
from tqdm import tqdm
import pyhdfs
from pyspark.sql import *
from pyspark.sql import functions as func
from pyspark.sql import SparkSession
from collections import Counter
import pandas as pd

c:\Users\Quang Khai\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


In [ ]:
df = spark.read.csv("hdfs://localhost:9000/big_data/coopmart_review_preprocess.csv", header=True, inferSchema=True)



In [ ]:
df = df.toPandas()
df.head(5)

,comment,stars
0,nhiều hàng hoá để lựa chọn thường xuyên có chư...,5
1,dịch vụ tốt,5
2,hôm nay ngày lúc tầm giờ giờ tôi tính tiền ở q...,1
3,nhân viên tính tiền thái độ khó chịu ra mặt,1
4,thật tuyệt,5


In [ ]:
checkpoint = "5CD-AI/Vietnamese-Sentiment-visobert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

from tqdm import tqdm  # Hiển thị tiến độ
from collections import Counter

def get_sentiments_in_batches(texts, batch_size=128):
    sentiment_mapping = {"POS": "Tích cực", "NEU": "Trung tính", "NEG": "Tiêu cực"}
    model.eval()
    all_sentiments = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i + batch_size]
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

            for pred in predictions:
                label = model.config.id2label[pred.item()]
                sentiment = sentiment_mapping.get(label, "Không xác định")
                all_sentiments.append(sentiment)

    return all_sentiments



In [ ]:
df['sentiment'] = get_sentiments_in_batches(df['comment'].tolist(), batch_size=128)


100%|██████████| 249/249 [41:31<00:00, 10.01s/it]


In [ ]:
df

,comment,stars,sentiment
0,nhiều hàng hoá để lựa chọn thường xuyên có chư...,5,Tích cực
1,dịch vụ tốt,5,Tích cực
2,hôm nay ngày lúc tầm giờ giờ tôi tính tiền ở q...,1,Tiêu cực
3,nhân viên tính tiền thái độ khó chịu ra mặt,1,Tiêu cực
4,thật tuyệt,5,Tích cực
...,...,...,...
31751,tốt,4,Tích cực
31752,tốt,4,Tích cực
31753,tốt,4,Tích cực
31754,tuyệt,4,Tích cực


In [ ]:
df.to_csv("coopmart_review_sentiment.csv",encoding='utf-8-sig' ,index=False)